In [1]:
import os
import sys
sys.path.insert(0, '..')
import numpy as np
import re

from metrics.kinetics_classify import *

In [2]:
### JOIN RESULTS ###

base_name = '/data/OnlineActionRecognition/outputs/causal_eval_kinetics_r50_i3d_nl_32x2/causal_eval_kinetics_r50_i3d_nl_32x2_4gpus_full'
# filenames = [base_name + '_{}.txt'.format(rank) for rank in range(8)]
# with open(base_name + '_joined.txt', 'w') as outfile:
#     for fid, fname in enumerate(filenames):
#         if os.path.exists(fname):
#             print(fname)
#             with open(fname) as infile:
#                 if fid > 0: # remove header
#                     infile.readline()
#                 for line in infile:
#                     outfile.write(line)
#         else:
#             outfile.write(video_metric.to_text())
            

In [4]:
with open(base_name + '_joined.txt', 'r') as file:
    header = file.readline()  # removing header
    text = sorted(file.readlines())
    
split_text = [t.replace('\n', '').split('|') for t in text if t != '\n']

In [4]:
labels = [int(st[1]) for st in split_text]
preds = [np.fromstring(st[2], dtype=int, sep=' ') for st in split_text]

In [5]:
labels[0], preds[0]

(104, array([104, 103, 199, 201, 355]))

In [6]:
top1_pred, top5_pred = get_top_predictions(preds, labels)

cls_acc1 = per_class_accuracy(top1_pred, labels)
report1 = classification_report(labels, top1_pred)

cls_acc5 = per_class_accuracy(top5_pred, labels)
report5 = classification_report(labels, top5_pred)

In [7]:
cls_acc1, cls_acc5

(0.6584932353221831, 0.8586824484186919)

In [8]:
re.sub('()_\\d{6}', '', split_text[10][0].strip()), split_text[10][0]

('driving_tractor/cn2OQeIdLcw.mp4',
 'driving_tractor/cn2OQeIdLcw.mp4_000099   ')

In [5]:
def divide_per_clip(split_text):
    clips_labels = []
    clips_prediction = []

    video_name = None
    for i, [video_path, label, pred] in enumerate(split_text):
        video = re.sub('()_\\d{6}', '', video_path.strip())
        if video != video_name or i == len(split_text):            
            # new video! But first, save old video
            if i > 0:
                video_labels = np.array(video_labels)
                video_prediction = np.array(video_prediction)
                
                num_frames = video_labels.shape[0]
                if num_frames < 10:
                    repeat_ids = np.round(np.linspace(0, num_frames - 1, 10)).astype(int)
                    video_labels = video_labels[repeat_ids]
                    video_prediction = video_prediction[repeat_ids]
                    
                clips_labels.append(video_labels)
                clips_prediction.append(video_prediction)
                    
            # star new one
            video_labels = []
            video_prediction = []
            video_name = video
            
        video_labels.append(int(label))
        video_prediction.append(np.fromstring(pred, dtype=int, sep=' '))
                
    return clips_labels, clips_prediction

In [10]:
len(labels), len(preds), len(np.hstack(labels)), len(np.vstack(preds))

(4655415, 4655415, 4655415, 4655415)

In [10]:
clip_labels, clip_pred = divide_per_clip(split_text)

In [7]:
len(clip_labels), len(clip_pred), len(np.hstack(clip_labels)), len(np.vstack(clip_pred))

(19678, 19678, 4655395, 4655395)

In [19]:
a = np.array_split(clip_labels[0], 10)
print(len(clip_labels[0]), len(a), a[0].shape, a[-1].shape)

b = np.array_split(clip_pred[0], 10)
print(len(clip_pred[0]), len(b), b[0].shape, b[-1].shape)

224 10 (23,) (22,)
224 10 (23, 5) (22, 5)


In [11]:
p_clip_labels = np.array([np.array_split(cl, 10) for cl in clip_labels])
p_clip_pred = np.array([np.array_split(cp, 10) for cp in clip_pred])

In [8]:
p_clip_labels.shape, p_clip_pred.shape

((19681, 10), (19681, 10))

In [12]:
percentage_labels = np.array([np.hstack(p_clip_labels[:,i]) for i in range(10)])
percentage_pred = np.array([np.vstack(p_clip_pred[:,i]) for i in range(10)])

In [14]:
for i, (p_labels, p_preds) in enumerate(zip(percentage_labels, percentage_pred)):
    top1_pred, top5_pred = get_top_predictions(p_preds, p_labels)

    cls_acc1 = per_class_accuracy(top1_pred, p_labels)
    cls_acc5 = per_class_accuracy(top5_pred, p_labels)
    print(i, cls_acc1, cls_acc5)

0 0.6350344431211492 0.8377650000658439
1 0.648033278110845 0.8504658228912407
2 0.6630772470201423 0.8620011322999497
3 0.6769041684487519 0.8745746650674004
4 0.678585180379324 0.8765110942162605
5 0.6780193731736717 0.8757074372901695
6 0.673837737693527 0.8720288702098982
7 0.6583584195089679 0.8589666744829388
8 0.6425492918499305 0.8456209133251477
9 0.631131913474855 0.8336249385207087


In [15]:
percentage_labels = np.array([np.hstack(np.hstack(p_clip_labels[:,:i])) for i in range(1, 11)])
percentage_pred = np.array([np.vstack(np.hstack(p_clip_pred[:,:i])) for i in range(1, 11)])

for i, (p_labels, p_preds) in enumerate(zip(percentage_labels, percentage_pred)):
    top1_pred, top5_pred = get_top_predictions(p_preds, p_labels)

    cls_acc1 = per_class_accuracy(top1_pred, p_labels)
    cls_acc5 = per_class_accuracy(top5_pred, p_labels)
    print(i, cls_acc1, cls_acc5)

0 0.6350344431211492 0.8377650000658439
1 0.6414932551292537 0.8440774941343215
2 0.6486538114064561 0.8500245764859587
3 0.6556873167878388 0.8561371993746958
4 0.660248638153106 0.8601957004562467
5 0.6631822096293005 0.8627574695776156
6 0.6646891688709102 0.8640683339034129
7 0.6639061171364224 0.8634371327859034
8 0.6615574420091217 0.8614776021891071
9 0.6585451062554735 0.8587210007858067


In [60]:
d = np.vstack(np.hstack(p_clip_pred[:,:2]))
e = np.vstack(np.hstack(p_clip_pred[:,:1]))
f = np.vstack(np.hstack(p_clip_pred[:,:3]))
g = np.vstack(np.hstack(p_clip_pred[:,:4]))
d.shape, e.shape, f.shape, g.shape

((942148, 5), (473872, 5), (1410021, 5), (1877517, 5))

In [68]:
d = np.hstack(np.hstack(p_clip_labels[:,:2]))
e = np.hstack(np.hstack(p_clip_labels[:,:1]))
f = np.hstack(np.hstack(p_clip_labels[:,:3]))
g = np.hstack(np.hstack(p_clip_labels[:,:4]))
d.shape, e.shape, f.shape, g.shape

((942148,), (473872,), (1410021,), (1877517,))

In [64]:
p_clip_pred[:,0][0].shape, p_clip_pred[:,1][0].shape, p_clip_pred[:,2].shape

((23, 5), (23, 5), (19681,))

In [65]:
p_clip_labels[:,0][0].shape, p_clip_labels[:,1].shape, p_clip_labels[:,2].shape

((23,), (19681,), (19681,))

In [54]:
f[0].shape, len(f), e[0].shape

((), 1410021, ())

In [67]:
p_clip_labels[:,:2].shape

(19681, 2)